# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import pandas as pd 
import nltk 
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin

nltk.download(['wordnet', 'punkt', 'stopwords','averaged_perceptron_tagger'])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName2',engine)  
#X = df.message.values #  Message Column
#Y = df.iloc[:, 4:]    # Classification label
X = df['message']
Y = df[df.columns[4:]]

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
    # Tokenize text  
    tokens = word_tokenize(text)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)), 
                ('tfidf', TfidfTransformer()), 
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Fit 
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def plot_scores(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))  

In [9]:
y_pred = pipeline.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.31      0.13      0.18      1502
          1       0.78      0.92      0.84      5085

avg / total       0.67      0.74      0.69      6587

Feature 2: request
             precision    recall  f1-score   support

          0       0.83      0.98      0.90      5451
          1       0.41      0.07      0.12      1136

avg / total       0.76      0.82      0.77      6587

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6558
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6587

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.59      0.84      0.69      3848
          1       0.45      0.18      0.26      2739

avg / total       0.53      0.57      0.51      6587

Feature 5: medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe7204ec620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid = parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Fit the first tuned model
cv.fit(X_train, y_train)

# Predicting using the first tuned model 
y_pred = cv.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.34      0.06      0.10      1502
          1       0.78      0.97      0.86      5085

avg / total       0.68      0.76      0.69      6587

Feature 2: request
             precision    recall  f1-score   support

          0       0.83      0.99      0.90      5451
          1       0.40      0.03      0.06      1136

avg / total       0.76      0.82      0.76      6587

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6558
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6587

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.59      0.86      0.70      3848
          1       0.45      0.16      0.24      2739

avg / total       0.53      0.57      0.51      6587

Feature 5: medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [14]:
def new_model_pipeline():
    new_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    return new_pipeline

In [15]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

new_model = new_model_pipeline()

# Fit the second tuned model
new_model.fit(X_train, y_train)

# Predicting using the second tuned model 
y_pred = new_model.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.33      0.12      0.17      1593
          1       0.77      0.93      0.84      4994

avg / total       0.66      0.73      0.68      6587

Feature 2: request
             precision    recall  f1-score   support

          0       0.84      0.98      0.91      5499
          1       0.43      0.08      0.13      1088

avg / total       0.77      0.83      0.78      6587

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6564
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6587

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.60      0.82      0.69      3901
          1       0.45      0.21      0.28      2686

avg / total       0.54      0.57      0.53      6587

Feature 5: medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.